# 4. Escalas, Eixos e Legendas

Codificação visual &ndash; mapear dados para variáveis visuais como posição, tamanho, forma ou cor &ndash; é o coração pulsante da visualização de dados. O mecanismo que realiza esse mapeamento é o *scale* (escala): uma função que recebe um valor de dado como input (o *domain* \[domínio\] da escala) e retorna um valor visual, como a posição de um píxel ou uma cor RGB, como output (o *range* \[intervalo\] da escala). Obviamente, uma visualização é inútil se ninguém for capaz de compreender o que ela transmite! Além dos elementos gráficos principais, um gráfico precisa de elementos de referência, ou *guides* (guias), que permitem aos leitores decodificar a representação. Guias como *axes* (eixos, que visualizam escalas com intervalos espaciais) e *legends* (legendas, que visualizam escalas com intervalos de cor, tamanho ou forma), são componentes essenciais para uma visualização eficaz de dados!

Nesse notebook, exploraremos as opções que o Altair oferece para suportar designs personalizados de mapeamentos de escala, eixos e legendas, utilizando um exemplo sobre a eficácia de remédios antibióticos.


_Esse notebook é parte de [_data visualization curriculum_](https://github.com/uwdata/visualization-curriculum) (currículo de visualização de dados)._

In [3]:
import pandas as pd
import altair as alt

## 4.1. Dados de Antibióticos

Após a segunda guerra mundial, antibióticos foram considerados "medicações milagrosas", uma vez que eram um remédio fácil para doenças antes consideradas incuráveis. Para identificar qual antibiótico era mais eficaz contra cada infecção bacteriana, foi coletado o desempenho dos três antibióticos mais populares em 16 espécies de bactérias.

Usaremos a base de dados de antibióticos da [_coleção vega-datasets_](https://github.com/vega/vega-datasets). Nos exemplos abaixo, passaremos o URL diretamente para o Altair:

In [4]:
antibiotics = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/burtin.json'

Primeiro, podemos carregar os dados com o Pandas para visualizar o _dataset_ na íntegra e nos familiarizar com os campos disponíveis:

In [5]:
pd.read_json(antibiotics)

,Bacteria,Penicillin,Streptomycin,Neomycin,Gram_Staining,Genus
0,Aerobacter aerogenes,870.000,1.00,1.600,negative,other
1,Bacillus anthracis,0.001,0.01,0.007,positive,other
2,Brucella abortus,1.000,2.00,0.020,negative,other
3,Diplococcus pneumoniae,0.005,11.00,10.000,positive,other
4,Escherichia coli,100.000,0.40,0.100,negative,other
5,Klebsiella pneumoniae,850.000,1.20,1.000,negative,other
6,Mycobacterium tuberculosis,800.000,5.00,2.000,negative,other
7,Proteus vulgaris,3.000,0.10,0.100,negative,other
8,Pseudomonas aeruginosa,850.000,2.00,0.400,negative,other
9,Salmonella (Eberthella) typhosa,1.000,0.40,0.008,negative,Salmonella


Os valores numéricos na tabela indicam a [concentração inibitória mínima (CIM)](https://en.wikipedia.org/wiki/Minimum_inhibitory_concentration), uma medida da eficácia do antibiótico, que representa a concentração de antibiótico (em microgramas por mililitro) necessária para prevenir o crescimento _in vitro_. A reação das bactérias a um procedimento chamado [coloração de Gram](https://en.wikipedia.org/wiki/Gram_stain) é descrita pelo campo nominal `Gram_Staining`. Bactérias que ficam azuis escuras ou violetas são Gram-positivas. Caso contrário, são Gram-negativas.

À medida que examinamos diferentes visualizações deste _dataset_, pergunte a si mesmo: O que podemos aprender sobre a eficácia relativa dos antibióticos? O que podemos aprender sobre as espécies bacterianas com base em suas respostas aos antibióticos?

## 4.2 Configurando Eixos e Escalas

### 4.2.1 Plotando Resistência Antibiótica: Ajustando o Tipo de Escala

Vamos começar observando um gráfico de pontos simples da CIM (Concentração Inibitória Mínima) para a Neomicina.

In [6]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q')
)

alt.Chart(...)

_Podemos observar que os valores de CIM abrangem ordens de magnitude: a maioria dos pontos se agrupa à esquerda, com alguns outliers grandes à direita._

Por padrão, o Altair usa um mapeamento `linear` entre os valores do domínio (domain, CIM) e do intervalo (range, pixels). Para obter uma visão geral melhor dos dados, podemos aplicar uma transformação de escala diferente.

Para alterar o tipo de escala, definiremos o atributo `scale`, usando o método `alt.Scale` e o parâmetro `type`.

Aqui está o resultado ao usar um tipo de escala de raiz quadrada (`sqrt`). Agora, as distâncias na escala de pixels correspondem à raiz quadrada das distâncias no domínio dos dados.

In [7]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale=alt.Scale(type='sqrt'))
)

alt.Chart(...)

Os pontos à esquerda agora estão melhor diferenciados, mas ainda observamos uma forte distorção.

Vamos tentar usar uma [escala logarítmica](https://en.wikipedia.org/wiki/Logarithmic_scale)(`log`) em vez disso:

In [8]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale=alt.Scale(type='log'))
)

alt.Chart(...)

_Agora os dados estão muito mais distribuídos uniformemente, e podemos ver as grandes diferenças nas concentrações necessárias para diferentes bactérias._

Em uma escala linear padrão, uma distância visual (em pixels) de 10 unidades pode corresponder a uma *adição* de 10 unidades no domínio dos dados. Uma transformação logarítmica mapeia entre multiplicação e adição, de forma que `log(u) + log(v) = log(u*v)`. Como resultado, em uma escala logarítmica, uma distância visual de 10 unidades corresponde a uma *multiplicação* por 10 unidades no domínio dos dados, assumindo uma base logarítmica de 10. A escala `log` acima usa, por padrão, o logaritmo de base 10, mas podemos ajustar isso fornecendo um parâmetro `base` à escala.

### 4.2.2 Estilizando de Eixo

Dosagens mais baixas indicam maior eficácia. No entanto, algumas pessoas podem esperar que valores "melhores" estejam "acima e à direita" em um gráfico. Se quisermos atender a essa convenção, podemos inverter o eixo para codificar a "efetividade" como uma escala de CIM invertida.

Para fazer isso, podemos definir a propriedade `sort` do encoding como `'descending'`:

In [9]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'))
)

alt.Chart(...)

*Infelizmente, o eixo está começando a ficar um pouco confuso: estamos plotando os dados em uma escala logarítmica, na direção inversa e sem uma indicação clara de quais são as nossas unidades!*

Vamos adicionar um título de eixo mais informativo: usaremos a propriedade `title` do encoding para fornecer o texto do título desejado:

In [10]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomicina CIM(μg/ml, escala logarítmica invertida)')
)

alt.Chart(...)

Muito melhor!

Por padrão, o Altair posiciona o eixo x na parte inferior do gráfico. Para alterar esses padrões, podemos adicionar um atributo `axis` com `orient='top'`:

In [11]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          axis=alt.Axis(orient='top'),
          title='Neomicina CIM(μg/ml, escala logarítmica invertida)')
)

alt.Chart(...)

Da mesma forma, o eixo y tem, por padrão, uma orientação à esquerda (`'left'`), mas pode ser configurado para a direita (`'right'`).



### 4.2.3 Comparando Antibióticos: Ajustando Linhas de Grade, Contagem de Marcas e Dimensionamento

_Como a neomicina se compara a outros antibióticos, como a estreptomicina e a penicilina?_

Para começar a responder essa pergunta, podemos criar gráficos de dispersão, adicionando uma codificação no eixo y para outro antibiótico que espelhe o design do nosso eixo x para a neomicina.

In [12]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Streptomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Streptomycin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

_Podemos ver que a neomicina e a estreptomicina parecem altamente correlacionadas, pois as cepas bacterianas respondem de forma semelhante a ambos os antibióticos._

Vamos prosseguir e comparar a neomicina com a penicilina:

In [13]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Penicillin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

_Agora vemos uma resposta mais diferenciada: algumas bactérias respondem bem à neomicina, mas não à penicilina, e vice-versa!_

Embora esse gráfico seja útil, podemos melhorá-lo. Os eixos x e y usam as mesmas unidades, mas possuem extensões diferentes (a largura do gráfico é maior que a altura) e domínios distintos (0,001 a 100 para o eixo x e 0,001 a 1.000 para o eixo y).

Vamos equalizar os eixos: podemos adicionar configurações explícitas de `width` e `height` para o gráfico e especificar domínios correspondentes usando a propriedade de escala `domain`.

In [14]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='Penicillin MIC (μg/ml, reverse log scale)')
).properties(width=250, height=250)

alt.Chart(...)

_O gráfico resultante está mais equilibrado e menos propenso a sutis interpretações errôneas!_

No entanto, as linhas de grade agora estão bastante densas. Se quisermos removê-las completamente, podemos adicionar `grid=False` ao atributo `axis`. Mas e se, em vez disso, quisermos reduzir o número de marcas de escala, por exemplo, incluindo linhas de grade apenas para cada ordem de magnitude?

Para alterar o número de marcas, podemos especificar a propriedade `tickCount` como um valor alvo para um objeto `Axis`. O `tickCount` é tratado como uma *sugestão* pelo Altair, considerando outros fatores, como o uso de intervalos bem distribuídos e fáceis de interpretar. Podemos não obter *exatamente* o número de marcas solicitado, mas teremos algo próximo.

In [15]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)')
).properties(width=250, height=250)

alt.Chart(...)

Ao definir `tickCount` como 5, conseguimos o efeito desejado.

Os pontos do nosso gráfico de dispersão parecem um pouco pequenos. Vamos alterar o tamanho padrão ajustando a propriedade `size` da marca de círculo. Esse valor de size representa a área da marca em pixels.

In [16]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'), 
).properties(width=250, height=250)

alt.Chart(...)

Aqui, definimos a área da marca de círculo para 80 pixels. _Ajuste esse valor conforme necessário!_

## 4.3 Configurando Legendas de Cores

### 4.3.1 Cor por Coloração de Gram

_Acima, vimos que a neomicina é mais eficaz para algumas bactérias, enquanto a penicilina funciona melhor para outras. Mas como podemos saber qual antibiótico usar se não conhecemos a espécie específica da bactéria? A coloração de Gram serve como um diagnóstico para diferenciar classes de bactérias!_

Vamos codificar `Gram_Staining` no canal `color` como um tipo de dado nominal:

In [17]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N')
).properties(width=250, height=250)

alt.Chart(...)

_Podemos ver que as bactérias Gram-positivas parecem ser mais suscetíveis à penicilina, enquanto a neomicina é mais eficaz contra bactérias Gram-negativas!_

O esquema de cores acima foi escolhido automaticamente para fornecer cores perceptualmente distintas para comparações nominais (igual ou diferente). No entanto, podemos querer personalizar as cores utilizadas. Nesse caso, a coloração de Gram resulta em [cores físicas características: rosa para Gram-negativas e roxo para Gram-positivas](https://en.wikipedia.org/wiki/Gram_stain#/media/File:Gram_stain_01.jpg).

Vamos usar essas cores especificando um mapeamento explícito da escala do domínio (`domain`) dos dados para o intervalo de `color`:

In [18]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple'])
    )
).properties(width=250, height=250)

alt.Chart(...)

Por padrão, as legendas são posicionadas no lado direito do gráfico. Assim como nos eixos, podemos alterar a orientação da legenda usando o parâmetro `orient`:

In [19]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=alt.Legend(orient='left')
    )
).properties(width=250, height=250)

alt.Chart(...)

Também podemos remover completamente a legenda especificando `legend=None`:

In [20]:

alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=None
    )
).properties(width=250, height=250)

alt.Chart(...)

### 4.3.2 Cor por Espécie

_Até agora, consideramos a eficácia dos antibióticos. Mudemos de perspectiva e façamos uma pergunta diferente: o que a resposta aos antibióticos pode nos ensinar sobre as diferentes espécies de bactérias?_

Para começar, vamos codificar `Bacteria` (um campo de dados nominal) usando o canal `color`:

In [21]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:N')
).properties(width=250, height=250)

alt.Chart(...)

_O resultado é de certa forma uma bagunça!_ Há tantas bactérias únicas que o Altair começa a repetir cores de sua paleta padrão de 10 cores para valores nominais.

Para usar cores personalizadas, podemos atualizar a propriedade `scale` da codificação de cor. Uma opção é fornecer valores de escala explícitos para `domain` e `range`, indicando mapeamentos de cores precisos por valor, como fizemos anteriormente para a coloração de Gram. Outra opção é usar um esquema de cores alternativo. O Altair inclui uma variedade de esquemas de cores integrados. Para uma lista completa, consulte [documentação de esquemas de cores do Vega](https://vega.github.io/vega/docs/schemes/#reference).

Vamos experimentar mudar para um esquema de 20 cores embutido, `tableau20`, e configurá-lo usando a propriedade `scheme` de escala.

In [22]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

_Agora temos uma cor única para cada bactéria, mas o gráfico ainda está confuso. Entre outros problemas, a codificação não leva em conta bactérias que pertencem ao mesmo gênero. No gráfico acima, as duas cepas diferentes de Salmonella possuem matizes muito distintas (verde-azulado e rosa), apesar de serem parentes biológicos._

Para tentar um esquema diferente, também podemos alterar o data type (tipo de dado) de nominal para ordinal. O esquema ordinal padrão usa tons de azul, variando de claro a escuro:

In [23]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:O')
).properties(width=250, height=250)

alt.Chart(...)

_Alguns desses tons de azul podem ser difíceis de distinguir._

Para obter cores mais diferenciadas, podemos experimentar alternativas ao esquema de cores padrão `blues`. O esquema `viridis` varia tanto em matiz quanto em luminância:

In [24]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:O',
          scale=alt.Scale(scheme='viridis'))
).properties(width=250, height=250)

alt.Chart(...)

_As bactérias do mesmo gênero agora possuem cores mais semelhantes do que antes, mas o gráfico ainda continua confuso. Há muitas cores, é difícil identificá-las com precisão na legenda, e duas bactérias podem ter cores semelhantes apesar de pertencerem a gêneros diferentes._

### 4.3.3 Cor por Gênero

Vamos tentar colorir por gênero em vez de por bactéria. Para isso, adicionaremos uma transformação `calculate` que divide o nome das bactérias nos espaços e pega a primeira palavra do array resultante. Em seguida, podemos codificar o campo `Genus` resultante usando o esquema de cores `tableau20`.

(Observe que o conjunto de dados de antibióticos inclui um campo `Genus` pré-calculado, mas iremos ignorá-lo aqui a fim de explorar mais as transformações de dados do Altair.)

In [25]:
alt.Chart(antibiotics).mark_circle(size=80).transform_calculate(
    Genus='split(datum.Bacteria, " ")[0]'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Genus:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

_Hmm... Embora os dados estejam melhor segregados por gênero, essa cacofonia de cores não parece particularmente útil._

_Se observarmos alguns dos gráficos anteriores com atenção, podemos ver que apenas um pequeno número de bactérias compartilha um gênero com outra bactéria: Salmonella, Staphylococcus e Streptococcus. Para focar nossa comparação, podemos adicionar cores apenas para esses valores de gênero repetidos._

Vamos adicionar outra transformação `calculate` que pega um nome de gênero, o mantém se for um dos valores repetidos e, caso contrário, usa a string `"Other"`.

Além disso, podemos adicionar codificações de cores personalizadas usando arrays `domain` e `range` explícitos para a codificação de cor `scale`.

In [26]:
alt.Chart(antibiotics).mark_circle(size=80).transform_calculate(
  Split='split(datum.Bacteria, " ")[0]'
).transform_calculate(
  Genus='indexof(["Salmonella", "Staphylococcus", "Streptococcus"], datum.Split) >= 0 ? datum.Split : "Other"'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Genus:N',
          scale=alt.Scale(
            domain=['Salmonella', 'Staphylococcus', 'Streptococcus', 'Other'],
            range=['rgb(76,120,168)', 'rgb(84,162,75)', 'rgb(228,87,86)', 'rgb(121,112,110)']
          ))
).properties(width=250, height=250)

alt.Chart(...)

_Nós temos agora um gráfico muito mais revelador, possibilitado pelas personalizações nos eixos e na legenda. Tome um momento para examinar o gráfico acima. Percebe alguma agrupação surpreendente?_

_A região no canto superior esquerdo possui um agrupamento de bactérias Streptococcus vermelhas, mas com uma bactéria "Other" (outras) em cinza ao lado delas. Enquanto isso, na parte central-direita, vemos outro Streptococcus vermelho localizado bem distante de seus "parentes". Podemos nós esperar que as bactérias do mesmo gênero (e, portanto, presumivelmente mais semelhantes geneticamente) fiquem agrupadas próximas umas das outras?_

Acontece que o _dataset_ subjacente de fato contém erros. O _dataset_ reflete as designações de espécies usadas no início dos anos 1950. No entanto, o consenso científico desde então foi revogado. Aquela ponto cinza no canto superior esquerdo? Agora é considerado um Streptococcus! Aquele ponto vermelho no meio-direita? Não é mais considerado um Streptococcus!

Claro, por si só, esse _dataset_ não justifica totalmente essas reclassificações. No entanto, os dados contêm evidências biológicas valiosas que ficaram despercebidas por décadas! A visualização, quando usada por um observador habilidoso e inquisitivo, pode ser uma ferramenta poderosa para a descoberta.

Este exemplo também reforça uma lição importante: **_sempre seja cético com seus dados!_**

### 4.3.4 Cor por Resposta Antibiótica

Nós tambem podemos usar o canal `color` para codificar valores quantitativos. No entanto, tenha em mente que, tipicamente, a cor não é tão eficaz para transmitir quantidades quanto codificações de posição ou tamanho!

Aqui está um mapa de calor básico dos valores MIC da penicilina para cada bactéria. Usaremos uma marca `rect` e ordenaremos as bactérias por valores MIC em ordem decrescente (da mais resistente à menos resistente):

In [27]:
alt.Chart(antibiotics).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending')
    ),
    alt.Color('Penicillin:Q')
)

alt.Chart(...)

Podemos melhorar ainda mais este gráfico combinando recursos que vimos até agora: uma escala com transformação logarítmica, uma mudança de orientação do eixo, um esquema de cores personalizado (`plasma`), ajuste da contagem de marcas no eixo e um título personalizado. Também utilizaremos opções de configuração para ajustar a posição do título do eixo e o alinhamento do título da legenda.

In [ ]:
alt.Chart(antibiotics).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(
        orient='right',     # posiciona o eixo no lado direito do gráfico
        titleX=7,           # define a posição x do título para 7 pixels à direita do gráfico
        titleY=-2,          # define a posição y do título para 2 pixels acima do gráfico
        titleAlign='left',  # alinha o título à esquerda
        titleAngle=0        # remove a rotação padrão do título
      )
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='Penicilina MIC (μg/ml)'
    )
)

alt.Chart(...)

Alternativamente, podemos remover o título do eixo completamente e usar a propriedade `title` de nível superior para adicionar um título ao gráfico inteiro:


In [ ]:
alt.Chart(antibiotics, title='Penicillin Resistance of Bacterial Strains').mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(orient='right', title=None)
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='Penicillin MIC (μg/ml)'
    )
).configure_title(
  anchor='start', # ancorar e alinhar o título à esquerda
  offset=5        # definir deslocamento do título em relação ao gráfico
)

alt.Chart(...)

## 4.4 Resumo

Integrando o que aprendemos até agora nos notebooks sobre codificações, transformações de dados e personalização, você agora deve estar preparado para fazer uma ampla variedade de gráficos estatísticos. Agora, você pode usar o Altair no dia a dia para explorar e comunicar dados!

Interessado em aprender mais sobre esse tópico?

- Comece com a [documentação de Personalização de Visualizações no Altair](https://altair-viz.github.io/user_guide/customization.html).
- Para uma discussão complementar sobre mapeamentos de escala, consulte ["Introdução a escala d3"](https://medium.com/@mbostock/introducing-d3-scale-61980c51545f).
- Para uma exploração mais aprofundada de todas as maneiras como eixos e legendas podem ser estilizados pela biblioteca subjacente Vega (que alimenta o Altair e o Vega-Lite), veja ["Um guia para guias: Eixos e Legendas em Vega"](https://beta.observablehq.com/@jheer/a-guide-to-guides-axes-legends-in-vega).
- Para uma história fascinante do _dataset_ de antibióticos, veja [Wainer &amp; Lysen's "That's Funny..."](https://www.americanscientist.org/article/thats-funny) no _American Scientist_.